# Movie Ratings
This notebook uses the Guidebox API to pull in currently available movies (e.g. from HBO or Amazon Prime) and the OMDB API to find ratings on those movies. It returns the sorted results in a dataframe to help you decide what to watch. Time to quit wasting so much time trying to find a decent movie!

TODO: add in ability to filter by genre

TODO: add in Netflix as a source

In [1]:
import os
import requests
import webbrowser

import numpy as np
import pandas as pd

region = 1 # United States
api_key = os.environ.get('guidebox_api_key')

guidebox_base = 'https://api-public.guidebox.com/v1.43/%s/%s' %(region,api_key)
omdb_base = 'http://www.omdbapi.com/?'


## parameters for guidebox request

In [2]:
# limits result set from guidebox, pre-sorted by popularity
limit_1 = 1
limit_2 = 250

# change source depending on need (options = ['amazon_prime','hbo'])
sources = 'amazon_prime'
platform = 'all'


## make guidebox request

In [3]:
# construct uri based on above parameters
uri = guidebox_base + '/movies/all/%s/%s/%s/%s' %(
str(limit_1),
str(limit_2),
sources,
platform)

r = requests.get(uri)
data = r.json()
if 'Error' in data.keys():
    print 'Error: %s' %data['Error']

## parse json

In [4]:
# capture data we want to include in the dataframe
guidebox_id = [movie['id'] for movie in data['results']]
title = [movie['title'] for movie in data['results']]
imdb_id = [movie['imdb'] for movie in data['results']]

df_data = [
    guidebox_id,
    title,
    imdb_id,
]

movie_df = pd.DataFrame(df_data).transpose()
movie_df.columns=['guidebox_id','title','imdb_id']


## get review information

In [10]:
# get imdb score & rotten tomatoes for movies guidebox list

# intialize dataframe
df_cols = [
    'imdb_id',
    'imdb_rating',
    'rt_rating',
    'metascore',
    'actors',
    'rt_review',
    'awards',
]
rating_df = pd.DataFrame(columns=df_cols)

index = 0
for i in imdb_id:
    uri = omdb_base + 'i=%s&tomatoes=true&r=json' %i
    r = requests.get(uri)
    movie_data = r.json()
    df_row = pd.Series({
        'imdb_id':i,
        'imdb_rating':movie_data['imdbRating'],
        'rt_rating':movie_data['tomatoUserMeter'],
        'metascore':movie_data['Metascore'],
        'actors':movie_data['Actors'],
        'rt_review':movie_data['tomatoConsensus'],
        'awards':movie_data['Awards'],
    })
    index+=1
    rating_df.loc[index]=df_row


{u'Plot': u'Tarzan, having acclimated to life in London, is called back to his former home in the jungle to investigate the activities at a mining encampment.', u'Rated': u'PG-13', u'tomatoImage': u'N/A', u'Title': u'The Legend of Tarzan', u'Ratings': [{u'Source': u'Internet Movie Database', u'Value': u'6.3/10'}, {u'Source': u'Rotten Tomatoes', u'Value': u'36%'}, {u'Source': u'Metacritic', u'Value': u'44/100'}], u'DVD': u'11 Oct 2016', u'tomatoMeter': u'N/A', u'Writer': u'Adam Cozad (screenplay), Craig Brewer (screenplay), Craig Brewer (story by), Adam Cozad (story by), Edgar Rice Burroughs (based on the "Tarzan" stories created by)', u'tomatoUserRating': u'N/A', u'Production': u'Lionsgate Pictures', u'Actors': u'Alexander Skarsg\xe5rd, Rory J. Saper, Christian Stevens, Christoph Waltz', u'tomatoFresh': u'N/A', u'Type': u'movie', u'imdbVotes': u'114,545', u'Website': u'http://legendoftarzan.com/', u'tomatoConsensus': u'N/A', u'Poster': u'https://images-na.ssl-images-amazon.com/images/M

## combine review information with original guidebox list

In [21]:
# combine dataframes
merged = movie_df.merge(
    rating_df,
    how='inner',
    left_on='imdb_id',
    right_on='imdb_id',
)

# drop rows where there is no rating from imdb or RT
merged = merged[(merged.rt_rating != 'N/A') | (merged.imdb_rating != 'N/A')]

# sort merged dataframe by rt_score
merged.sort_values(
    by='imdb_rating',
    ascending=False,
    inplace=True
)

merged.reset_index(
    inplace=True,
    drop=True,
)

# view top results
print 'Top movies available now on %s:' %sources
merged.head(50)

Top movies available now on amazon_prime:


,guidebox_id,title,imdb_id,imdb_rating,rt_rating,metascore,actors,rt_review,awards
0,112659,Interstellar,tt0816692,8.6,N/A,74,"Ellen Burstyn, Matthew McConaughey, Mackenzie ...",N/A,Won 1 Oscar. Another 41 wins & 142 nominations.
1,401,Life is Beautiful,tt0118799,8.6,N/A,59,"Roberto Benigni, Nicoletta Braschi, Giorgio Ca...",N/A,Won 3 Oscars. Another 66 wins & 51 nominations.
2,69183,Indiana Jones and the Raiders of the Lost Ark,tt0082971,8.5,N/A,85,"Harrison Ford, Karen Allen, Paul Freeman, Rona...",N/A,Won 4 Oscars. Another 30 wins & 23 nominations.
3,27738,American Beauty,tt0169547,8.4,N/A,86,"Kevin Spacey, Annette Bening, Thora Birch, Wes...",N/A,Won 5 Oscars. Another 103 wins & 96 nominations.
4,202,Good Will Hunting,tt0119217,8.3,N/A,70,"Matt Damon, Robin Williams, Ben Affleck, Stell...",N/A,Won 2 Oscars. Another 22 wins & 55 nominations.
5,34134,Indiana Jones and the Last Crusade,tt0097576,8.3,N/A,65,"Harrison Ford, Sean Connery, Denholm Elliott, ...",N/A,Won 1 Oscar. Another 6 wins & 20 nominations.
6,56419,Gone with the Wind,tt0031381,8.2,N/A,N/A,"Thomas Mitchell, Barbara O'Neil, Vivien Leigh,...",N/A,Won 8 Oscars. Another 10 wins & 9 nominations.
7,134210,Room,tt3170832,8.2,N/A,86,"Brie Larson, Jacob Tremblay, Sean Bridgers, We...",N/A,Won 1 Oscar. Another 102 wins & 131 nominations.
8,17662,The Truman Show,tt0120382,8.1,N/A,90,"Jim Carrey, Laura Linney, Noah Emmerich, Natas...",N/A,Nominated for 3 Oscars. Another 38 wins & 66 n...
9,76752,There Will Be Blood,tt0469494,8.1,N/A,92,"Daniel Day-Lewis, Martin Stringer, Matthew Bra...",N/A,Won 2 Oscars. Another 106 wins & 134 nominations.


## View Trailers (work in progress)

In [104]:
def view_trailer(guidebox_id):

    # {Base API URL} /movie/ {id} /videos/ {sources} / {limit 1} / {limit 2}
    uri = guidebox_base + '/movie/%s/videos/guidebox/1/1' %(
        guidebox_id,
    )

    # capture just the first free link
    r = requests.get(uri)
    trailer_data = r.json()
    trailer = trailer_data['results'][0]['free_web_sources'][0]['link']

    # open trailer
    webbrowser.open(trailer)

In [105]:
# try it out

view_trailer(17874)